In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
# drive.mount('/content/drive')
import os
skipDataCreate=False
skipDataCreate=True
%cd /content/drive/MyDrive/Final Project/model
os.listdir()

Mounted at /content/drive
/content/drive/MyDrive/Final Project/model


['generator.py',
 '__pycache__',
 'datacounter.ipynb',
 'resnet9.ipynb',
 'writer.py',
 'data',
 'data.csv',
 'data 1.csv',
 'data 2.csv',
 'data 3.csv',
 'bestParams_old',
 'bestParams.txt',
 'rnn.ipynb']

In [ ]:
# # for colab only
if not skipDataCreate:
  # if writer:
  #   del writer
# !rm data
  import writer
  writer.resizeFactor=1
  writer.how_many_orbits=20
  writer.generator.num_of_points=1000
  writer.csvWrite=True
  writer.mainFunc()
  del writer
  # writer.mainFunc()
else:
  print("Skipped")

# # print(os.getcwd())


Skipped


In [ ]:
# LSTM for international airline passengers problem with memory
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pdb; 

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
def LSTM_Reg(dataset,train_size=0.67,batch_size=1,train_epochs=100,per_train_epochs=5,plot=False,look_back = 3,verbose=1):
  # pre-prep the data
  scaler = MinMaxScaler(feature_range=(0, 1))
  dataset = scaler.fit_transform(dataset) # normalize
  # split into train and test sets
  train_size = int(len(dataset) * train_size)
  test_size = len(dataset) - train_size
  train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
  # reshape into X=t and Y=t+1
  trainX, trainY = create_dataset(train, look_back)
  testX, testY = create_dataset(test, look_back)
  # reshape input to be [samples, time steps, features]
  trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
  testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))
    
  model = Sequential()
  model.add(LSTM(40, batch_input_shape=(batch_size, look_back, 1), stateful=True))
  model.add(Dense(10))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')
  for i in range(train_epochs):
    if verbose:
      print("Superepoch %d of %d" % (i,train_epochs))
    model.fit(trainX, trainY, epochs=per_train_epochs, batch_size=batch_size, verbose=verbose, shuffle=False)
    model.reset_states()
  # make predictions
  trainPredict = model.predict(trainX, batch_size=batch_size)
  model.reset_states()
  testPredict = model.predict(testX, batch_size=batch_size)
  # invert predictions
  trainPredict = scaler.inverse_transform(trainPredict)
  trainY = scaler.inverse_transform([trainY])
  testPredict = scaler.inverse_transform(testPredict)
  testY = scaler.inverse_transform([testY])
  # calculate root mean squared error
  trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
  testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
  if verbose:
    print('Train Score: %.2f RMSE' % (trainScore))
    print('Test Score: %.2f RMSE' % (testScore))
  
  
  # shift train predictions for plotting

  if plot:
    # PLT
    trainPredictPlot = numpy.empty_like(dataset)
    trainPredictPlot[:, :] = numpy.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
    # shift test predictions for plotting
    testPredictPlot = numpy.empty_like(dataset)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
    # plot baseline and predictions
    plt.plot(scaler.inverse_transform(dataset))
    plt.plot(trainPredictPlot)
    plt.plot(testPredictPlot)
    plt.show()
  return testScore

In [ ]:
# fix random seed for reproducibility
numpy.random.seed(7)
xyz=["y","x","z"]

bestParams=()
maxScore=0
verbose=0
try:
  with open("bestParams.txt") as BC:
    BC=BC.readlines()
    maxScore=float(BC[1])
    BC=BC[0][1:-2].split(', ')
    bestParams=tuple(map(float,BC))
    print("Score file loaded, score is:")
    print(maxScore)
    print("Best parameters found for now are the following:")
    print(BC)
except IOError:
  print("Score file not found, starting from 0")

from datetime import datetime
today = datetime.now()
d4 = today.strftime("%b-%d-%Y_%H-%M-%S")
logfile = open(f"log_{d4}.txt","a+")

# iterate through various parameters to find the best ones
for per_train_epochs in range(1,9,2):
  for look_back in range(2,8):
    for train_epochs in range(1,10,1):
      for train_size in numpy.arange(0.65,0.95,0.05):        
        outerScore=0
        for j in range(1,4):
          curScore=0
          for i in range(1,4):
            # load the dataset with given column
            dataframe = read_csv(f'data {j}.csv', usecols=[i], engine='python')
            dataset = dataframe.values
            dataset = dataset.astype('float32')
            if verbose:
              print(f'{xyz[i-1]} score:')
            score = LSTM_Reg(dataset,per_train_epochs=per_train_epochs,look_back=look_back,train_epochs=train_epochs,train_size=float(train_size),verbose=verbose)
            if type(score)==type(None):
              score=0
            curScore += score          
          curScore/=3
          outerScore+=curScore
        outerScore/=3
        
        d4 = today.strftime("%b-%d-%Y_%H-%M-%S")
        logfile.write(f"{d4}\t|\t{outerScore}\t|\t{(per_train_epochs,look_back,train_epochs,train_size)}\n")
        if outerScore > maxScore:
          logfile.write(f"***\nCHANGE\n***")
          print(f'Old score ({maxScore}) => New Score ({outerScore})')
          maxScore=outerScore
          bestParams=(per_train_epochs,look_back,train_epochs,train_size)
          with open('bestParams.txt', 'w') as writefile:
            writefile.write(f"{bestParams}\n{maxScore}")

Score file loaded, score is:
0.1850603527152933
Best parameters found for now are the following:
['1', '2', '1', '0.65']
Old score (0.1850603527152933) => New Score (0.19842257428379828)


In [ ]:
# # print(f"best Params:\n{bestParams}")
# # print(f"best score:\n{maxScore}")
# with open('bestParams', 'w') as writefile:
#     writefile.write(f"{bestParams}\n{maxScore}")

In [ ]:
!ls
# 

 bestParams_old  'data 2.csv'	      generator.py		     rnn.ipynb
 bestParams.txt  'data 3.csv'	      log_May-10-2022_00-00-00.txt   writer.py
 data		  datacounter.ipynb   __pycache__
'data 1.csv'	  data.csv	      resnet9.ipynb


# New Section